# Assignment 3: Create a SageMaker pipeline
In this assignment you create an end-to-end ML workflow using [SageMaker Pipelines](https://aws.amazon.com/sagemaker/pipelines/).

Refer to the notebook [`03-sagemaker-pipeline.ipynb`](../03-sagemaker-pipeline.ipynb) for code snippets and a general guidance for the exercises in this assignment.

## Import packages

In [1]:
%pip install 'sagemaker[feature-processor]' --force-reinstall

  Using cached sagemaker-2.253.1-py3-none-any.whl.metadata (17 kB)
  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached boto3-1.40.61-py3-none-any.whl.metadata (6.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 75.6 MB/s  0:00:03:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 67.1 MB/s  0:00:00m0:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached botocore-1.40.61-py3-none-any.whl.metadata (5.7 kB)
  Using cached s3transfer-0.14.0-py3-none-any.whl.metadata (1.7 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Using cached sagemaker-2.253.1-py3-none-any.whl (1.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import pandas as pd
import json
import boto3
import pathlib
import io
import sagemaker
import mlflow
from time import gmtime, strftime, sleep
from sagemaker.deserializers import CSVDeserializer
from sagemaker.serializers import CSVSerializer
from importlib.metadata import version, PackageNotFoundError

from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import (
    ProcessingInput, 
    ProcessingOutput, 
    ScriptProcessor
)
from sagemaker.inputs import TrainingInput

from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import (
    ProcessingStep, 
    TrainingStep, 
    CreateModelStep,
    CacheConfig
)
from sagemaker.workflow.check_job_config import CheckJobConfig
from sagemaker.workflow.parameters import (
    ParameterInteger, 
    ParameterFloat, 
    ParameterString, 
    ParameterBoolean
)
from sagemaker.workflow.quality_check_step import (
    DataQualityCheckConfig,
    ModelQualityCheckConfig,
    QualityCheckStep,
)
from sagemaker.workflow.clarify_check_step import (
    ModelBiasCheckConfig, 
    ClarifyCheckStep, 
    ModelExplainabilityCheckConfig
)
from sagemaker import Model
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.conditions import (
    ConditionGreaterThan,
    ConditionGreaterThanOrEqualTo
)
from sagemaker.workflow.parallelism_config import ParallelismConfiguration
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import (
    Join,
    JsonGet
)
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)
from sagemaker.lambda_helper import Lambda

from sagemaker.model_metrics import (
    MetricsSource, 
    ModelMetrics, 
    FileSource
)
from sagemaker.drift_check_baselines import DriftCheckBaselines
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig 
from sagemaker.image_uris import retrieve
from sagemaker.workflow.function_step import step
from sagemaker.workflow.step_outputs import get_step
from sagemaker.model_monitor import DatasetFormat, model_monitoring
from IPython.display import HTML

(sagemaker.__version__, boto3.__version__, mlflow.__version__)

('2.253.1', '1.40.61', '2.22.2')

## Set constants
You need some fixed string literals to simplify coding. Create these literals here.

In [5]:
bucket_name = sagemaker.Session().default_bucket()
bucket_prefix = "from-idea-to-prod/xgboost" 

In [6]:
# Set names of pipeline objects
project = "from-idea-to-prod"
pipeline_name = f"{project}-pipeline"
pipeline_model_name = f"{project}-model-xgb"
model_package_group_name = f"{project}-model-group"
endpoint_config_name = f"{project}-endpoint-config"
endpoint_name = f"{project}-endpoint"

# Set instance types and counts
process_instance_type = "ml.c5.xlarge"
train_instance_count = 1
train_instance_type = "ml.m5.xlarge"

# Set S3 urls for processed data
output_s3_prefix = f"s3://{bucket_name}/{bucket_prefix}"
output_s3_url = f"{output_s3_prefix}/output"
train_s3_url = f"{output_s3_prefix}/train"
validation_s3_url = f"{output_s3_prefix}/validation"
test_s3_url = f"{output_s3_prefix}/test"
evaluation_s3_url = f"{output_s3_prefix}/evaluation"
baseline_s3_url = f"{output_s3_prefix}/baseline"
baseline_results_s3_url = f"{baseline_s3_url}/results"
prediction_baseline_s3_url = f"{output_s3_prefix}/prediction_baseline"
prediction_baseline_results_s3_url = f"{prediction_baseline_s3_url}/results"

## Exercise 1: Create pipeline
Follow the steps to create a SageMaker pipeline with your ML workflow:
- Setup pipeline [parameters](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#parameters)
- Build the pipeline [steps](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#steps)
- Construct the [pipeline](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.pipeline.Pipeline)
- [Upsert](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.pipeline.Pipeline.upsert) the pipeline

Configure outputs and inputs for each pipeline step.

In [22]:
#Jake, you will need to update these values later if you don't finish

input_s3_url = f"s3://{bucket_name}/{bucket_prefix}/input"
mlflow_arn = "arn:aws:sagemaker:us-east-2:284038851123:mlflow-tracking-server/mlflow-d-21qjmpg5nbfo-16-17-36-05"
model_approval_status = "PendingManualApproval"  # Or "Approved", depending on your usage


In [23]:
# Setup pipeline parameters
# Set S3 url for input dataset
# input_s3_url_param = ParameterString()
# Set processing instance type
process_instance_type_param = ParameterString(
    name="ProcessingInstanceType",
    default_value=process_instance_type,
)

# Set training instance type
train_instance_type_param = ParameterString(
    name="TrainingInstanceType",
    default_value=train_instance_type,
)

# Set model approval status for the model registry
model_approval_status_param = ParameterString(
    name="ModelApprovalStatus",
    default_value=model_approval_status
)

# Minimal threshold for model performance on the test dataset
test_score_threshold_param = ParameterFloat(
    name="TestScoreThreshold",
    default_value=0.75
)

# Parametrize the S3 url for input dataset
input_s3_url_param = ParameterString(
    name="InputDataUrl",
    default_value=input_s3_url,
)

# Model package group name
model_package_group_name_param = ParameterString(
    name="ModelPackageGroupName",
    default_value=model_package_group_name,
)

# MLflow tracking server ARN
tracking_server_arn_param = ParameterString(
    name="TrackingServerARN",
    default_value=mlflow_arn,
)

In [24]:
import sagemaker
role = sagemaker.get_execution_role()

In [25]:
# Processing step
sklearn_processor = SKLearnProcessor(
    framework_version="1.2-1",
    role=role,
    instance_type=process_instance_type_param,
    instance_count=1,
)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


In [26]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

processing_inputs=[
    ProcessingInput(
        source=input_s3_url,
        destination="/opt/ml/processing/input"
    )
]
processing_outputs=[
    ProcessingOutput(
        output_name="train",
        source="/opt/ml/processing/train"
    ),
    ProcessingOutput(
        output_name="test",
        source="/opt/ml/processing/test"
    )
]

In [27]:
from sagemaker.workflow.steps import ProcessingStep

step_process = ProcessingStep(
    name="PreprocessData",
    processor=sklearn_processor,
    inputs=processing_inputs,
    outputs=processing_outputs,
    code="pipeline_steps/preprocessing.py",
    job_arguments=[
        "--input-data", "/opt/ml/processing/input/data.csv",
        "--output-train", "/opt/ml/processing/train/train.csv",
        "--output-test", "/opt/ml/processing/test/test.csv",
        "--test-size", "0.2"
    ]
)

In [28]:
from sagemaker.workflow.steps import ProcessingStep

step_process = ProcessingStep(
    name="PreprocessData",
    processor=sklearn_processor,
    inputs=processing_inputs,
    outputs=processing_outputs,
    code="pipeline_steps/preprocessing.py",
    job_arguments=[
        "--input-data", "/opt/ml/processing/input/data.csv",
        "--output-train", "/opt/ml/processing/train/train.csv",
        "--output-test", "/opt/ml/processing/test/test.csv",
        "--test-size", "0.2"
    ]
)

In [29]:
# Python function code is in the local files
from pipeline_steps.preprocess import preprocess

In [ ]:
# Training step
# estimator = sagemaker.estimator.Estimator()

# estimator.set_hyperparameters()

# training_inputs = {}

# training_args = estimator.fit(training_inputs)

# step_train = TrainingStep()

In [31]:
from sagemaker.sklearn import SKLearn

estimator = SKLearn(
    entry_point="training/train.py",                  # your actual training script
    role=role,
    instance_type=train_instance_type_param, # pipeline parameter from earlier
    instance_count=1,
    framework_version="0.23-1",              # or update as needed
    output_path=f's3://{bucket_name}/{bucket_prefix}/'  # update to your bucket and folder
)

In [32]:
estimator.set_hyperparameters(
    n_estimators=100,          # Example, update to match your script!
    max_depth=5,
    random_state=42
    # add other hyperparameters as needed
)

In [33]:
training_inputs = {
    "train": step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
    "test": step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri
}


In [34]:
from sagemaker.workflow.steps import TrainingStep

step_train = TrainingStep(
    name="TrainModel",
    estimator=estimator,
    inputs={
        "train": training_inputs["train"],
        "test": training_inputs["test"]
    }
)

Create an executable Python script for the evaluation step

In [35]:
%%writefile evaluation_assignment.py
import json
import pathlib
import pickle as pkl
import tarfile
import joblib
import numpy as np
import pandas as pd
import xgboost as xgb
import datetime as dt
from sklearn.metrics import roc_curve, auc

def load_model(model_tar_path):
    # Untar the model.tar.gz and load model
    with tarfile.open(model_tar_path) as tar:
        member_list = tar.getnames()
        # Find model file (e.g. "model.pkl", "model.joblib", "xgboost-model")
        model_files = [m for m in member_list if m.endswith('.pkl') or m.endswith('.joblib')]
        if model_files:
            model_file = model_files[0]
            tar.extract(model_file, path="/tmp")  # extract to temp
            model_path = f"/tmp/{model_file}"
            if model_file.endswith('.pkl'):
                with open(model_path, 'rb') as f:
                    model = pkl.load(f)
            else:
                model = joblib.load(model_path)
            return model
        else:
            # For xgboost saver
            for m in member_list:
                if "xgboost" in m or m.endswith(".model"):
                    tar.extract(m, path="/tmp")
                    model_path = f"/tmp/{m}"
                    return xgb.Booster({'nthread':4}, model_file=model_path)
            raise ValueError("No model file found in tar")
    raise ValueError("No valid tar file available.")

def run_predictions(model, test_df):
    if hasattr(model, 'predict'):
        # sklearn/xgboost sklearn API
        preds = model.predict(test_df.drop("target", axis=1, errors="ignore"))
    else:
        # xgboost Booster API
        dmat = xgb.DMatrix(test_df.drop("target", axis=1, errors="ignore"))
        preds = model.predict(dmat)
    return preds

def evaluate_predictions(test_df, preds):
    # Use ROC-AUC as example, requires a "target" column
    y_true = test_df.get("target")
    if y_true is not None:
        fpr, tpr, thresholds = roc_curve(y_true, preds)
        score = auc(fpr, tpr)
        metrics = {
            "roc_auc": score,
            "timestamp": dt.datetime.now().isoformat()
        }
    else:
        metrics = {
            "detail": "Target column not found for evaluation."
        }
    return metrics

if __name__ == "__main__":
    # Paths would be passed as arguments in real pipeline code
    model_tar_path = "/opt/ml/processing/model/model.tar.gz"
    test_data_path = "/opt/ml/processing/test/test.csv"
    report_path = "/opt/ml/processing/evaluation/eval.json"

    print("Loading model...")
    model = load_model(model_tar_path)
    print("Loading test data...")
    test_df = pd.read_csv(test_data_path)
    print("Running predictions...")
    preds = run_predictions(model, test_df)
    print("Evaluating predictions...")
    metrics = evaluate_predictions(test_df, preds)
    print("Saving evaluation report...")
    pathlib.Path(report_path).parent.mkdir(parents=True, exist_ok=True)
    with open(report_path, "w") as f:
        json.dump(metrics, f)
    print("Done.")

Overwriting evaluation_assignment.py


Use this script to setup a [ScriptProcessor](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ScriptProcessor) object. Don't forget to pass the [PropertyFile](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.properties.PropertyFile) where the evaluation script outputs the metrics to the [ProcessingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep) constructor.

In [ ]:
# Evaluation step
# script_processor = ScriptProcessor()

# eval_inputs=[]

# eval_outputs=[]

# eval_args = script_processor.run()

# evaluation_report = PropertyFile()

# step_eval = ProcessingStep()

In [37]:
from sagemaker.processing import ScriptProcessor

script_processor = ScriptProcessor(
    image_uri="683313688378.dkr.ecr.us-east-2.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3"
,                        # Put correct image URI (e.g. sklearn/python/xgboost)
    command=["python3"],
    instance_type=process_instance_type_param, # pipeline param if needed
    instance_count=1,
    role=role
)


In [38]:
from sagemaker.processing import ProcessingInput

eval_inputs = [
    ProcessingInput(
        source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
        destination="/opt/ml/processing/model"
    ),
    ProcessingInput(
        source=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
        destination="/opt/ml/processing/test"
    )
]


In [39]:
from sagemaker.processing import ProcessingOutput

eval_outputs = [
    ProcessingOutput(
        output_name="evaluation",
        source="/opt/ml/processing/evaluation"
    )
]


In [41]:
from sagemaker.workflow.properties import PropertyFile

evaluation_report = PropertyFile(
    name="EvaluationReport",
    output_name="evaluation",
    path="eval.json"
)

In [42]:
from sagemaker.workflow.steps import ProcessingStep

step_eval = ProcessingStep(
    name="EvaluateModel",
    processor=script_processor,
    inputs=eval_inputs,
    outputs=eval_outputs,
    code="evaluation_assignment.py",     # Path to your evaluation script
    property_files=[evaluation_report]
)


Use a model artifact from the training step and a property file from the evaluation step to create a [Model](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model) and [ModelMetrics](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_metrics.ModelMetrics) objects. Use these objects to construct a [ModelStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.model_step.ModelStep).

In [ ]:
# Register step
# model = Model()

# model_metrics = ModelMetrics()

# register_args = model.register()

# step_register = ModelStep()

In [47]:
model = Model(
    image_uri=estimator.training_image_uri(),  # from estimator/config
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=sagemaker.Session(),
    role=role
)

In [52]:
from sagemaker.model_metrics import ModelMetrics, MetricsSource

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=step_eval.properties.ProcessingOutputConfig.Outputs[0].S3Output.S3Uri,
        content_type="application/json"
    )
)

In [69]:
print("Type of model_package_group_name_param.default_value:", type(model_package_group_name_param.default_value))
print("Value of model_package_group_name_param.default_value:", model_package_group_name_param.default_value)


Type of model_package_group_name_param.default_value: <class 'str'>
Value of model_package_group_name_param.default_value: from-idea-to-prod-model-group


In [70]:
from sagemaker.workflow.parameters import ParameterString

model_package_group_name_param = ParameterString(
    name="ModelPackageGroupName",
    default_value="your-actual-model-package-group-name"
)


In [73]:
register_args = model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    model_package_group_name=model_package_group_name_param.default_value,  # <-- for pipeline context
    model_metrics=model_metrics,
    approval_status=model_approval_status_param.default_value
)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 register_args = model.register(                                                              │
│   2 │   content_types=["text/csv"],                                                              │
│   3 │   response_types=["text/csv"],                                                             │
│   4 │   model_package_group_name=model_package_group_name_param.default_value,  # <-- for pi     │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/workflow/pipeline_context.py:346 in wrapper    │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/model.py:605 in register                       │
│                                                                                                  │
│    602 │   │   │   model_card=model_card,                                                        │
│    603 │   │   │   model_life_cycle=model_life_cycle,                                            │
│    604 │   │   )                                                                                 │
│ ❱  605 │   │   model_package = self.sagemaker_session.create_model_package_from_containers(      │
│    606 │   │   │   **model_pkg_args                                                              │
│    607 │   │   )                                                                                 │
│    608 │   │   if isinstance(self.sagemaker_session, PipelineSession):                           │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/session.py:4653 in                             │
│ create_model_package_from_containers                                                             │
│                                                                                                  │
│   4650 │   │   │   │   return self.sagemaker_client.update_model_package(**update_source_uri_ar  │
│   4651 │   │   │   return self.sagemaker_client.create_model_package(**request)                  │
│   4652 │   │                                                                                     │
│ ❱ 4653 │   │   return self._intercept_create_request(                                            │
│   4654 │   │   │   model_pkg_request, submit, self.create_model_package_from_containers.__name_  │
│   4655 │   │   )                                                                                 │
│   4656                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/session.p

In [ ]:
from sagemaker.workflow.steps import ModelStep

step_register = ModelStep(
    name="RegisterTrainedModel",
    step_args=register_args
)


Add a [FailStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.fail_step.FailStep) to stop the pipeline execution if the model performance metric doesn't meet the specified threshold. 

In [ ]:
# Fail step
# step_fail = FailStep()

Use [Conditions](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#conditions) and `JsonGet` to construct a condition and the condition step.

In [ ]:
# Condition step
# cond_lte = ConditionGreaterThan()

# step_cond = ConditionStep()

Use step array to create a [Pipeline](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.pipeline.Pipeline) object.

In [ ]:
# Create pipeline
# pipeline = Pipeline()

In [ ]:
# Create a new or update existing Pipeline
# pipeline.upsert(role_arn=sm_role)

In [ ]:
# Print the pipeline definition
# pipeline_definition = json.loads(pipeline.describe()['PipelineDefinition'])
# pipeline_definition

## Exercise 2: Execute the pipeline

In [ ]:
# Start an execution
# execution = pipeline.start()

In [ ]:
# Un-comment this call if you want the notebook to wait until the pipeline's execution finished
# Execution time of this pipeline is about 13 minutes
# execution.wait()

In [ ]:
# List execution steps
# execution.list_steps()

## Continue with the assignment 4
Navigate to the [assignment 4](04-assignment-sagemaker-project.ipynb) notebook.